# exploring 2017 test and training sets

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('BPI Challenge 2017-training.csv')
test = pd.read_csv('BPI Challenge 2017-test.csv')

In [3]:
test_time = test['event time:timestamp'].copy()
train_time = train['event time:timestamp'].copy()

In [4]:
train.head()

,eventID,case LoanGoal,case ApplicationType,case concept:name,case RequestedAmount,event Action,event org:resource,event concept:name,event EventOrigin,event EventID,event lifecycle:transition,event time:timestamp
0,0,Existing loan takeover,New credit,Application_652823628,20000.0,Created,User_1,A_Create Application,Application,Application_652823628,complete,01-01-2016 10:51:15.304
1,1,Existing loan takeover,New credit,Application_652823628,20000.0,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,01-01-2016 10:51:15.352
2,2,Existing loan takeover,New credit,Application_652823628,20000.0,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,01-01-2016 10:51:15.774
3,3,Existing loan takeover,New credit,Application_652823628,20000.0,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,01-01-2016 10:52:36.392
4,4,Existing loan takeover,New credit,Application_652823628,20000.0,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,01-01-2016 10:52:36.403


In [5]:
test.head()

,eventID,case LoanGoal,case ApplicationType,case concept:name,case RequestedAmount,event Action,event org:resource,event concept:name,event EventOrigin,event EventID,event lifecycle:transition,event time:timestamp
0,108263240630272,Existing loan takeover,New credit,Application_424064154,17250.0,Created,User_1,A_Create Application,Application,Application_424064154,complete,19-10-2016 01:42:39.265
1,108263240630273,Existing loan takeover,New credit,Application_424064154,17250.0,statechange,User_1,A_Submitted,Application,ApplState_875814280,complete,19-10-2016 01:42:40.100
2,108263240630274,Existing loan takeover,New credit,Application_424064154,17250.0,Created,User_1,W_Handle leads,Workflow,Workitem_1964338518,schedule,19-10-2016 01:42:40.711
3,108267535597568,"Other, see explanation",New credit,Application_608775916,10000.0,Created,User_1,A_Create Application,Application,Application_608775916,complete,19-10-2016 04:16:17.680
4,108267535597569,"Other, see explanation",New credit,Application_608775916,10000.0,statechange,User_1,A_Submitted,Application,ApplState_1860095003,complete,19-10-2016 04:16:18.446


In [6]:
#dt_test = pd.to_datetime(test_time)
#dt_train = pd.to_datetime(train_time)

In [7]:
# dt_test

In [8]:
# dt_train

# Set 2012

In [150]:
train = pd.read_csv('BPI_Challenge_2012-training.csv')
test = pd.read_csv('BPI_Challenge_2012-test.csv')

In [3]:
train.head()

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256


In [4]:
train.rename(columns={'case concept:name':'case_name'}, inplace=True)

In [12]:
cases = list(train.case_name.unique())

In [13]:
# dicts={}

# for case in cases:
    
#     ran = len(train[train['case_name']==case])
#     ID = train[['event concept:name']][train['case_name']==case]
    
#     for i in range(ran):
#         dicts[i]= ID.iloc[i][0]

# method github

In [5]:
df_sort = train.set_index(train.groupby('case_name').cumcount(), append = True)

In [6]:
df_new = df_sort.groupby(level=1)["event concept:name"].transform(lambda x: x.value_counts().index[0])
df_new = df_new.to_frame()
df_new = df_new.rename(columns = {'event concept:name' : "common activity"})
df_sort = df_sort.join(df_new["common activity"])

In [7]:
df_sort = df_sort.rename(columns = {'event time:timestamp' : "timestamp"})
df_sort['timestamp'] = pd.to_datetime(df_sort.timestamp)

In [8]:
df_times = df_sort.groupby('case_name')["timestamp"].transform(lambda x: x.diff())
df_times = df_times.to_frame()
df_times = df_times.rename(columns = {'timestamp' : "time_diff"})

In [9]:
df_sort = df_sort.join(df_times["time_diff"])

In [16]:
df_sort[df_sort['case_name']==173688].head()

,,eventID,case_name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,timestamp,common activity,time_diff
0,0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,2011-01-10 00:38:44.546,A_SUBMITTED,NaT
1,1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,2011-01-10 00:38:44.880,A_PARTLYSUBMITTED,00:00:00.334000
2,2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,2011-01-10 00:39:37.906,W_Afhandelen leads,00:00:53.026000
3,3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,2011-01-10 00:39:38.875,W_Afhandelen leads,00:00:00.969000
89,4,4,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,START,2011-01-10 11:36:46.437,W_Completeren aanvraag,10:57:07.562000


# Function for adding the next event

In [174]:
def make_event_pred(df_train, df_test):
    
    '''Parameters: df_train=training set
                   df_test=test set'''
    
    df_sort = df_train.set_index(df_train.groupby('case concept:name').cumcount(), append = True)
    df_new = df_sort.groupby(level=1)["event concept:name"].transform(lambda x: x.value_counts().index[0])
    df_new = df_new.to_frame()
    df_new = df_new.rename(columns = {'event concept:name' : "common activity"})
    
    # Here we can implement the average time to add later in the dataframe
    
    max_list = df_new.groupby(level=1).apply(max).copy()
        
    test = df_test.set_index(df_test.groupby('case concept:name').cumcount(), append = True)
    
    for i, df in test.groupby('case concept:name'):
        length = len(df)
        test = test.append({'case concept:name':int(i),
                            'event concept:name': max_list.loc[length, 'common activity']}, ignore_index=True)
                            # 'event time:timestamp': }, ignore_index=True
                            # here we can add the time of the added new event
    test = test.set_index(test.groupby('case concept:name').cumcount(), append = True)
    
    return test

# Test stuff for the function

In [148]:
pos = df_new.groupby(level=1).apply(max).copy()

In [167]:
df_test = test.set_index(test.groupby('case concept:name').cumcount(), append = True)
df_test.head()

,,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,44964012621824,206324,2012-02-03T17:17:11.047+01:00,2500,A_SUBMITTED,COMPLETE,03-02-2012 17:17:11.047
1,1,44964012621825,206324,2012-02-03T17:17:11.047+01:00,2500,A_PARTLYSUBMITTED,COMPLETE,03-02-2012 17:17:11.323
2,2,44964012621826,206324,2012-02-03T17:17:11.047+01:00,2500,A_DECLINED,COMPLETE,03-02-2012 17:17:42.964
3,0,44968307589120,206327,2012-02-03T17:23:41.949+01:00,6000,A_SUBMITTED,COMPLETE,03-02-2012 17:23:41.949
4,1,44968307589121,206327,2012-02-03T17:23:41.949+01:00,6000,A_PARTLYSUBMITTED,COMPLETE,03-02-2012 17:23:42.504


In [168]:
for i, df in df_test.groupby('case concept:name'):
    length = len(df)
    df_test = df_test.append({'case concept:name':int(i),
                              'event concept:name': pos.loc[length, 'common activity']},ignore_index=True)
df_test = df_test.set_index(df_test.groupby('case concept:name').cumcount(), append = True)
    #print(length)

In [169]:
df_test[df_test['case concept:name']==206324]

,,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,4.496401e+13,206324,2012-02-03T17:17:11.047+01:00,2500.0,A_SUBMITTED,COMPLETE,03-02-2012 17:17:11.047
1,1,4.496401e+13,206324,2012-02-03T17:17:11.047+01:00,2500.0,A_PARTLYSUBMITTED,COMPLETE,03-02-2012 17:17:11.323
2,2,4.496401e+13,206324,2012-02-03T17:17:11.047+01:00,2500.0,A_DECLINED,COMPLETE,03-02-2012 17:17:42.964
47823,3,NaN,206324,NaN,NaN,W_Afhandelen leads,NaN,NaN


# Testing function

In [175]:
train = pd.read_csv('BPI_Challenge_2012-training.csv')
test = pd.read_csv('BPI_Challenge_2012-test.csv')

In [176]:
saved = make_event_pred(train, test)

In [179]:
saved[saved['case concept:name']==214367]

,,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
38029,0,5.619106e+13,214367,2012-02-29T23:28:41.098+01:00,500.0,A_SUBMITTED,COMPLETE,29-02-2012 23:28:41.098
38030,1,5.619106e+13,214367,2012-02-29T23:28:41.098+01:00,500.0,A_PARTLYSUBMITTED,COMPLETE,29-02-2012 23:28:41.198
38034,2,5.619106e+13,214367,2012-02-29T23:28:41.098+01:00,500.0,A_DECLINED,COMPLETE,29-02-2012 23:29:21.958
50437,3,NaN,214367,NaN,NaN,W_Afhandelen leads,NaN,NaN


In [178]:
saved

,,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,4.496401e+13,206324,2012-02-03T17:17:11.047+01:00,2500.0,A_SUBMITTED,COMPLETE,03-02-2012 17:17:11.047
1,1,4.496401e+13,206324,2012-02-03T17:17:11.047+01:00,2500.0,A_PARTLYSUBMITTED,COMPLETE,03-02-2012 17:17:11.323
2,2,4.496401e+13,206324,2012-02-03T17:17:11.047+01:00,2500.0,A_DECLINED,COMPLETE,03-02-2012 17:17:42.964
3,0,4.496831e+13,206327,2012-02-03T17:23:41.949+01:00,6000.0,A_SUBMITTED,COMPLETE,03-02-2012 17:23:41.949
4,1,4.496831e+13,206327,2012-02-03T17:23:41.949+01:00,6000.0,A_PARTLYSUBMITTED,COMPLETE,03-02-2012 17:23:42.504
...,...,...,...,...,...,...,...,...
50436,29,NaN,214364,NaN,NaN,W_Nabellen offertes,NaN,NaN
50437,3,NaN,214367,NaN,NaN,W_Afhandelen leads,NaN,NaN
50438,6,NaN,214370,NaN,NaN,W_Completeren aanvraag,NaN,NaN
50439,17,NaN,214373,NaN,NaN,W_Nabellen offertes,NaN,NaN
